<a href="https://colab.research.google.com/github/vadlasushma/Sentiment_Analysis_legal/blob/main/data_Augmentation_with_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
!mkdir results

mkdir: cannot create directory ‘results’: File exists


In [84]:
import pandas as pd
import re
import numpy as np



In [85]:
def data_clean(test_list) :
  # while("" in test_list) :
  #   test_list.remove("")
  length = len(test_list)
  tested_list = []
  for i in range(length) :
    x=test_list[i].replace('_1', '')
    x=x.replace('_2','')
    x=x.replace('_3','')
    x=x.replace('_4','')
    x=x.replace('_5','')
    tested_list.append(x)
  test_list=tested_list
  return test_list  

In [86]:
def aspects_and_sentiments(aspectArray,sentimentArray) :
  aspects_together = [] 
  sentiments_together = []
  for i in range(len(aspectArray)):
    aspect=None
    sentiment=None
    aspect = aspectArray[i].replace('[', '').replace(']', '')
    PartyLabel = sentimentArray[i].replace('[', '').replace(']', '')
    aspects_together.append(aspect.split(','))
    sentiments_together.append(PartyLabel.split(','))
  # aspects_together = list(np.concatenate(aspects_together).flat)
  # sentiments_together = list(np.concatenate(sentiments_together).flat)
  aspects_together[0] = data_clean(aspects_together[0])
  aspects_together[1] = data_clean(aspects_together[1])
  # sentiments_together = data_clean(sentiments_together)
  return aspects_together,sentiments_together   


In [87]:
def writes_to_file(sentences,aspects,sentiments) :
  for i in range(len(sentences)) :
    temp = []
    temp.append(sentences[i])
    # file.write("\n")
    temp.append(aspects[i])
    # file.write("\n")
    temp.append(sentiments[i])
    # file.write("\n")
    csvwriter.writerow(temp)
    for j in range(len(sentences)):
      temp = []
      if i!=j :
        temp.append(sentences[i])
        # file.write("\n")
        temp.append(aspects[j])
        # file.write("\n")
        temp.append(sentiments[j])
        # file.write("\n")
        csvwriter.writerow(temp)


In [88]:

def mask_data(sentence,aspect) :
  new_sentences = []
  n1 = len(aspect)
  n2=0
  
  for i in range(len(aspect)) :
    regex1 = re.compile(r'\b' + aspect[i] + '\'s' + r'\b', re.IGNORECASE)
    matched1 = regex1.search(sentence)
    regex2 = re.compile(r'\b' + aspect[i] + r'\b', re.IGNORECASE)
    matched2 = regex2.search(sentence)
    # print(regex1, matched1)
    # print(regex2,matched2)
    if matched1 is not None and bool(matched1.group()):
      n2=n2+1
      new_sentences.append(re.sub(r'\b' + aspect[i] + '\'s' + r'\b', "$T$", sentence, count=1, flags=re.I))
            

    else:
      if matched2 is not None and bool(matched2.group()):
        n2=n2+1
        newSentence = re.sub(r'\b' + aspect[i] + r'\b', "$T$", sentence, count=1, flags=re.I)
        new_sentences.append(newSentence)
                
  # if(n2!=n1) :
    # error=error+1
    # print(aspect)
    # print(n1,n2,"not equal")
  return new_sentences   

In [89]:
import csv
df = pd.read_csv("/content/drive/My Drive/Datasets/SigmaLaw-ABSA_edited_dataset.csv",encoding='cp1252')
# file_name = open("results/SigmaLaw-ABSA-duplicated.txt", "w+")

list_aspects = None
list_sentiments = None
count=1
n1=0
n2=0
error =0
with open("results/SigmaLaw-ABSA-augmented.csv", "w+") as file:
    fields = ['Sentence', 'Aspect', 'Sentiment']
    csvwriter = csv.writer(file) 
    # writing the fields 
    csvwriter.writerow(fields)
    # w = csv.writer(file)
    
    for index, row in df.iterrows():
        count=count+1
        masked_sentences = []
        print(count)
        sentence = row["Sentence"]
        sentimentList = row["Sentiment"]
        sentimentArray = sentimentList.split('],[')
        aspectList = row["Party"]
        aspectArray = aspectList.split('],[')
        list_aspects,list_sentiments = aspects_and_sentiments(aspectArray,sentimentArray)
        print(list_aspects,list_sentiments)
        if(list_aspects[0]!=['']) :
          masked_sentences.append(mask_data(sentence,list_aspects[0]))
        else :
          masked_sentences.append([''])
        if(list_aspects[1]!=['']):
          masked_sentences.append(mask_data(sentence,list_aspects[1]))
        else :
          masked_sentences.append([''])
        print(masked_sentences)

        if(masked_sentences[0]!=['']) :
          writes_to_file(masked_sentences[0],list_aspects[0],list_sentiments[0])
        if(masked_sentences[1]!=['']):
          writes_to_file(masked_sentences[1],list_aspects[1],list_sentiments[1])
        # writes_to_file(masked_sentences,list_aspects,list_sentiments)

Streaming output truncated to the last 5000 lines.
[['petitioner', 'he', 'he'], ['Government']] [['-1', ' -1', '-1'], ['1']]
[['It reasoned that $T$ "did not dispute" the Government theory that he "helped shape the message" of the allegedly fraudulent presentations made at the analyst conference, and therefore rejected the District Court conclusion that the jury had "acquitted [petitioner] on the groun[d] that he did not participate in the fraud."', 'It reasoned that petitioner "did not dispute" the Government theory that $T$ "helped shape the message" of the allegedly fraudulent presentations made at the analyst conference, and therefore rejected the District Court conclusion that the jury had "acquitted [petitioner] on the groun[d] that he did not participate in the fraud."', 'It reasoned that petitioner "did not dispute" the Government theory that $T$ "helped shape the message" of the allegedly fraudulent presentations made at the analyst conference, and therefore rejected the Distr

In [90]:
import pandas as pd
file_name = "results/SigmaLaw-ABSA-augmented.csv"
file_name_output = "results/SigmaLaw-ABSA-augmented_withoutdupes.csv"

df = pd.read_csv(file_name)

# Notes:
# - the `subset=None` means that every column is used 
#    to determine if two rows are different; to change that specify
#    the columns as an array
# - the `inplace=True` means that the data structure is changed and
#   the duplicate rows are gone  
df.drop_duplicates(subset=None, inplace=True)

# Write the results to a different file
df.to_csv(file_name_output, index=False)